# PCAP_encoder 完整演示教程

本 Notebook 演示了 PCAP_encoder 的完整工作流程，使用真实的 CIC-IDS2017 数据集进行演示。

## 📋 目录

### 第一部分：数据生成与准备
1. 环境设置和依赖检查
2. 加载真实网络流量数据 (CIC-IDS2017)
3. 数据可视化与分析
4. 字节到十六进制转换演示

### 第二部分：端到端流程
5. 构建模型输入（问题+上下文）
6. T5 分词器编码
7. 加载 T5 编码器
8. 推理与分类预测
9. 结果分析与总结

---

## ⚙️ 运行说明

- 按顺序执行每个 Cell
- 每个 Cell 都有详细的中文注释
- 可以独立查看每个步骤的输出结果


# 第一部分：数据生成与准备

## 1. 环境设置和依赖检查

In [23]:
# 导入必要的库
import os
import sys
from pathlib import Path
from typing import Tuple, List, Dict, Optional

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 设置显示选项
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 100)

print("✅ 基础库导入成功")
print(f"   - NumPy version: {np.__version__}")
print(f"   - Pandas version: {pd.__version__}")

# 检查深度学习库
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   运行设备: {device}")
except ImportError:
    print("❌ PyTorch 未安装")

try:
    import transformers
    print(f"✅ Transformers version: {transformers.__version__}")
except ImportError:
    print("❌ Transformers 未安装 - 部分功能将不可用")

try:
    import pyarrow
    print(f"✅ PyArrow version: {pyarrow.__version__}")
except ImportError:
    print("⚠️  PyArrow 未安装 - Parquet 文件支持可能不可用")

✅ 基础库导入成功
   - NumPy version: 1.26.4
   - Pandas version: 2.2.3
✅ PyTorch version: 2.2.2+cu118
   运行设备: cuda
✅ Transformers version: 4.39.1
✅ PyArrow version: 22.0.0


## 2. 加载真实网络流量数据 (CIC-IDS2017)

我们将从 `data/CIC-IDS2017/Payload-Bytes/` 目录加载真实的 CIC-IDS2017 数据集。
为了确保演示包含多种流量类型，我们将合并以下四个文件：
- `Payload_Bytes_File_1.parquet` (主要包含 BENIGN)
- `Payload_Bytes_File_3.parquet` (包含攻击流量)
- `Payload_Bytes_File_12.parquet` (包含攻击流量)
- `Payload_Bytes_File_18.parquet` (包含攻击流量)

我们将从合并后的数据中随机采样 500 个样本进行演示。


In [25]:
# 加载真实数据并创建不重叠的训练/测试划分
data_paths = [
    "../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_1.parquet",
    "../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_17.parquet",
    "../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_12.parquet",
    "../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_18.parquet"
]

print(f"⏳ 正在从 {len(data_paths)} 个文件加载真实数据...")

# 读取并合并 Parquet 文件
dfs = []
for path in data_paths:
    if os.path.exists(path):
        print(f"   - 加载: {path}")
        dfs.append(pd.read_parquet(path))
    else:
        print(f"   ⚠️  文件不存在: {path}")

if not dfs:
    raise FileNotFoundError("未找到任何数据文件，请检查路径。")

df_full = pd.concat(dfs, ignore_index=True)
payload_cols = [c for c in df_full.columns if c.startswith("payload_byte_")]

# 设定采样规模并保持训练/测试不重叠
target_total = 1000  # 最大总采样数
target_train = 500
target_test = 500

rng = np.random.default_rng(42)
available = len(df_full)
n_total = min(target_total, available)
if n_total < target_train + target_test:
    target_train = min(target_train, n_total // 2)
    target_test = n_total - target_train

perm_indices = rng.permutation(available)[:n_total]
train_idx = perm_indices[:target_train]
test_idx = perm_indices[target_train:target_train + target_test]

df_train = df_full.iloc[train_idx].reset_index(drop=True)
df_test = df_full.iloc[test_idx].reset_index(drop=True)

# 处理缺失值 (将 <NA> 填充为 0)
for split_df in (df_train, df_test):
    split_df[payload_cols] = split_df[payload_cols].fillna(0).astype(np.uint8)

# 主演示数据使用训练集
df = df_train.copy()

print(f"✅ 成功加载数据并划分训练/测试！")
print(f"   - 原始文件总行数: {len(df_full)}")
print(f"   - 训练集大小: {len(df_train)}")
print(f"   - 测试集大小: {len(df_test)}")
print(f"   - 训练集标签: {df_train['attack_label'].unique().tolist()}")
print(f"   - 测试集标签: {df_test['attack_label'].unique().tolist()}")

⏳ 正在从 4 个文件加载真实数据...
   - 加载: ../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_1.parquet
   - 加载: ../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_17.parquet
   - 加载: ../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_12.parquet
   - 加载: ../data/CIC-IDS2017/Payload-Bytes/Payload_Bytes_File_18.parquet
✅ 成功加载数据并划分训练/测试！
   - 原始文件总行数: 5364165
   - 训练集大小: 500
   - 测试集大小: 500
   - 训练集标签: ['BENIGN', 'DDoS']
   - 测试集标签: ['BENIGN', 'DDoS']


In [26]:
# 保存采样后的数据集
save_dir = Path("../data/demo")
save_dir.mkdir(parents=True, exist_ok=True)
save_path = save_dir / "sampled_500.parquet"

print(f"💾 正在保存采样后的数据集到: {save_path}")
df.to_parquet(save_path, index=False)
print("✅ 保存成功！")


💾 正在保存采样后的数据集到: ../data/demo/sampled_500.parquet
✅ 保存成功！


In [27]:
# 查看已保存的采样数据概况
saved_path = Path("../data/demo/sampled_500.parquet")
if saved_path.exists():
    df_saved = pd.read_parquet(saved_path)
    print(f"✅ 已读取保存的文件: {saved_path}")
    print(f"   - 形状: {df_saved.shape}")
    print(f"   - 标签列唯一值: {df_saved['attack_label'].unique().tolist()}")
    print()
    print("📋 前 5 条记录（仅显示标签与前 4 个字节列）：")
    display_cols = ['attack_label'] + [c for c in df_saved.columns if c.startswith('payload_byte_')][:4]
    print(df_saved[display_cols].head().to_string(index=False))
else:
    print(f"⚠️ 未找到文件: {saved_path}")


✅ 已读取保存的文件: ../data/demo/sampled_500.parquet
   - 形状: (500, 1509)
   - 标签列唯一值: ['BENIGN', 'DDoS']

📋 前 5 条记录（仅显示标签与前 4 个字节列）：
attack_label  payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4
      BENIGN             123             137             152             103
      BENIGN             110              26             102             176
      BENIGN             230             215             125             197
      BENIGN             172             159             114             102
        DDoS             114             111             109              32


In [28]:
# 保存测试集（已与训练集划分不重叠）
test_save_dir = Path("../data/demo")
test_save_dir.mkdir(parents=True, exist_ok=True)
test_save_path = test_save_dir / "sampled_500_test.parquet"

print(f"💾 使用预划分的测试集保存到: {test_save_path}")
df_test.to_parquet(test_save_path, index=False)
print("✅ 测试集保存成功！")

# 简要查看测试集标签分布
print("标签分布（测试集）:")
label_counts_test = df_test["attack_label"].value_counts()
for label, count in label_counts_test.items():
    pct = count / len(df_test) * 100
    bar = "█" * int(pct / 5)
    print(f"   {label:15s}: {count:3d} ({pct:5.1f}%) {bar}")

💾 使用预划分的测试集保存到: ../data/demo/sampled_500_test.parquet
✅ 测试集保存成功！
标签分布（测试集）:
   BENIGN         : 466 ( 93.2%) ██████████████████
   DDoS           :  34 (  6.8%) █


## 3. 数据可视化与分析

查看加载的真实数据摘要、标签分布和样本内容。

In [29]:
# 数据摘要
print("="*70)
print("📊 数据摘要")
print("="*70)
print(f"总样本数: {len(df)}")
print(f"字节列数: {len([c for c in df.columns if c.startswith('payload_byte_')])}")
print()

# 标签分布
print("📈 标签分布:")
label_counts = df["attack_label"].value_counts()
for label, count in label_counts.items():
    pct = count / len(df) * 100
    bar = "█" * int(pct / 5)
    print(f"   {label:15s}: {count:3d} ({pct:5.1f}%) {bar}")

print()
print("="*70)
print("📋 数据预览（前 5 条，部分列）")
print("="*70)
display_cols = ["attack_label"] + [f"payload_byte_{i}" for i in range(1, 9)]
print(df[display_cols].head().to_string(index=True))

# 统计信息
print()
print("="*70)
print("📊 数据统计")
print("="*70)
payload_cols = [c for c in df.columns if c.startswith("payload_byte_")]
print(df[payload_cols].describe().iloc[:3].to_string())  # 只显示 count, mean, std

📊 数据摘要
总样本数: 500
字节列数: 1500

📈 标签分布:
   BENIGN         : 474 ( 94.8%) ██████████████████
   DDoS           :  26 (  5.2%) █

📋 数据预览（前 5 条，部分列）
  attack_label  payload_byte_1  payload_byte_2  payload_byte_3  payload_byte_4  payload_byte_5  payload_byte_6  payload_byte_7  payload_byte_8
0       BENIGN             123             137             152             103              91             207             144              98
1       BENIGN             110              26             102             176              96              32              82              23
2       BENIGN             230             215             125             197             205             165             243             237
3       BENIGN             172             159             114             102              51               6             192             210
4         DDoS             114             111             109              32             116             104             101              10

## 4. 字节到十六进制转换演示

这是 PCAP_encoder 的关键步骤：将网络数据包的字节数据转换为十六进制字符串，使其可以被 NLP 模型（T5）处理。

In [30]:
print("="*70)
print("🔄 Payload 字节 → 十六进制字符串转换示例")
print("="*70)
print()

payload_cols = [c for c in df.columns if c.startswith("payload_byte_")][:16]  # 只取前16字节

n_examples = 3
for i in range(min(n_examples, len(df))):
    row = df.iloc[i]
    label = row["attack_label"]
    bytes_data = [int(row[col]) for col in payload_cols]
    
    # 原始字节（前8个）
    bytes_str = " ".join(f"{b:3d}" for b in bytes_data[:8])
    
    # 十六进制字符串
    hex_str = "".join(f"{b:02x}" for b in bytes_data)
    
    # 每4字符分组（PCAP_encoder 的 "every4" 格式）
    hex_grouped = " ".join(hex_str[j:j+4] for j in range(0, len(hex_str), 4))
    
    # ASCII 表示（如果可打印）
    ascii_str = ""
    for b in bytes_data[:8]:
        if 32 <= b <= 126:  # 可打印 ASCII 范围
            ascii_str += chr(b)
        else:
            ascii_str += "."
    
    print(f"样本 {i+1} - {label}")
    print(f"├─ 原始字节 (前8个): [{bytes_str} ...]")
    print(f"├─ 十六进制字符串:    {hex_grouped}")
    print(f"└─ ASCII 表示:        [{ascii_str}]")
    print()

print("💡 说明:")
print("   - 十六进制格式 'every4' 表示每4个字符用空格分隔")
print("   - 这种格式让 T5 模型可以像处理自然语言一样处理网络数据包")

🔄 Payload 字节 → 十六进制字符串转换示例

样本 1 - BENIGN
├─ 原始字节 (前8个): [123 137 152 103  91 207 144  98 ...]
├─ 十六进制字符串:    7b89 9867 5bcf 9062 13b1 bf05 939b c088
└─ ASCII 表示:        [{..g[..b]

样本 2 - BENIGN
├─ 原始字节 (前8个): [110  26 102 176  96  32  82  23 ...]
├─ 十六进制字符串:    6e1a 66b0 6020 5217 94c0 8e31 56fc ec30
└─ ASCII 表示:        [n.f.` R.]

样本 3 - BENIGN
├─ 原始字节 (前8个): [230 215 125 197 205 165 243 237 ...]
├─ 十六进制字符串:    e6d7 7dc5 cda5 f3ed 80ff b7ff 562a 15ad
└─ ASCII 表示:        [..}.....]

💡 说明:
   - 十六进制格式 'every4' 表示每4个字符用空格分隔
   - 这种格式让 T5 模型可以像处理自然语言一样处理网络数据包


# 第二部分：端到端流程

## 5. 构建模型输入（问题 + 上下文）

T5 模型使用 Text-to-Text 格式，我们需要构造：
- **Question**: 任务描述（如 "Classify the network packet"）
- **Context**: 十六进制表示的数据包

In [31]:
def build_text_fields(df, question="Classify the network packet", n_samples=10):
    """
    将数据转换为模型输入格式
    
    Returns:
        questions: 问题列表
        contexts: 上下文列表（十六进制字符串）
        labels: 标签数组
        label_names: 标签名称列表
    """
    # 取样本子集（用于演示）
    if len(df) > n_samples:
        df_subset = df.sample(n=n_samples, random_state=42).reset_index(drop=True)
    else:
        df_subset = df
    
    # 获取所有 payload 列
    payload_cols = sorted(
        [c for c in df_subset.columns if c.startswith("payload_byte_")],
        key=lambda x: int(x.split("_")[-1])
    )
    
    # 转换为十六进制字符串
    byte_array = df_subset[payload_cols].to_numpy(dtype=np.uint16)
    contexts = []
    for row in byte_array:
        hex_str = "".join(f"{int(b):02x}" for b in row)
        # 每4字符分组（PCAP_encoder 格式）
        hex_grouped = " ".join(hex_str[i:i+4] for i in range(0, len(hex_str), 4))
        contexts.append(hex_grouped)
    
    # 构造问题
    questions = [question] * len(contexts)
    
    # 标签编码
    labels, uniques = pd.factorize(df_subset["attack_label"], sort=True)
    
    return questions, contexts, labels.astype(np.int64), uniques.tolist(), df_subset

# 构建文本字段
print("🔄 将数据转换为 T5 模型输入格式...")
n_demo_samples = 500  # 演示使用500个样本
questions, contexts, labels, label_names, df_subset = build_text_fields(
    df, n_samples=n_demo_samples
)

print(f"✅ 转换完成！")
print(f"   - 样本数量: {len(questions)}")
print(f"   - 问题模板: \"{questions[0]}\"")
print(f"   - 上下文长度: {len(contexts[0])} 字符")
print(f"   - 标签类别: {len(label_names)} 类")
print(f"   - 标签映射: {dict(enumerate(label_names))}")

🔄 将数据转换为 T5 模型输入格式...
✅ 转换完成！
   - 样本数量: 500
   - 问题模板: "Classify the network packet"
   - 上下文长度: 3749 字符
   - 标签类别: 2 类
   - 标签映射: {0: 'BENIGN', 1: 'DDoS'}


In [32]:
# 展示样本转换详情
print("="*70)
print("📋 样本转换详情")
print("="*70)
print()

payload_cols = sorted(
    [c for c in df_subset.columns if c.startswith("payload_byte_")],
    key=lambda x: int(x.split("_")[-1])
)[:8]  # 只取前8字节

for i in range(min(3, len(df_subset))):
    row = df_subset.iloc[i]
    print(f"┌─ 样本 {i+1} ─────────────────────────────────────────────")
    print(f"│ 标签: {row['attack_label']} (编码: {labels[i]})")
    
    bytes_vals = [int(row[col]) for col in payload_cols]
    print(f"│ 原始字节 (前8): {bytes_vals}")
    
    print(f"│ 问题: {questions[i]}")
    print(f"│ 上下文: {contexts[i][:70]}...")
    print(f"└{'─' * 59}")
    print()

📋 样本转换详情

┌─ 样本 1 ─────────────────────────────────────────────
│ 标签: BENIGN (编码: 0)
│ 原始字节 (前8): [123, 137, 152, 103, 91, 207, 144, 98]
│ 问题: Classify the network packet
│ 上下文: 7b89 9867 5bcf 9062 13b1 bf05 939b c088 523a 9e4e 35ef e4e1 31e7 1265 ...
└───────────────────────────────────────────────────────────

┌─ 样本 2 ─────────────────────────────────────────────
│ 标签: BENIGN (编码: 0)
│ 原始字节 (前8): [110, 26, 102, 176, 96, 32, 82, 23]
│ 问题: Classify the network packet
│ 上下文: 6e1a 66b0 6020 5217 94c0 8e31 56fc ec30 6bb3 8b7a d087 bf4e bbc7 b434 ...
└───────────────────────────────────────────────────────────

┌─ 样本 3 ─────────────────────────────────────────────
│ 标签: BENIGN (编码: 0)
│ 原始字节 (前8): [230, 215, 125, 197, 205, 165, 243, 237]
│ 问题: Classify the network packet
│ 上下文: e6d7 7dc5 cda5 f3ed 80ff b7ff 562a 15ad 8a79 f610 0070 edbd df06 5896 ...
└───────────────────────────────────────────────────────────



## 6. T5 分词器编码

使用 T5 的分词器将文本输入转换为 token ID，这是模型能够理解的数字表示。

In [33]:
try:
    from transformers import T5TokenizerFast
    from pathlib import Path
    
    print("⏳ 正在加载 T5 分词器...")
    
    # 检查预训练权重使用的模型版本
    # PCAP_encoder 预训练权重基于 t5-base (隐藏维度=768, 12层)
    weights_path = Path("../models/weights.pth")
    
    if weights_path.exists():
        model_name = "t5-base"  # 匹配预训练权重的模型架构
        print(f"📦 检测到预训练权重，使用 {model_name} 架构")
    else:
        model_name = "t5-small"  # 无预训练权重时使用小模型加快演示
        print(f"⚠️ 未找到预训练权重，使用 {model_name} 进行演示")
    
    tokenizer = T5TokenizerFast.from_pretrained(model_name)
    
    print(f"✅ 分词器加载成功: {model_name}")
    print()
    
    # 编码示例
    print("🔤 分词器编码示例:")
    sample_input = f"question: {questions[0][:30]}... context: {contexts[0][:40]}..."
    tokens = tokenizer.tokenize(sample_input)[:15]
    print(f"   输入: {sample_input}")
    print(f"   Token数量(前15): {tokens}")
    print()
    
    # 批量编码
    print("🔄 批量编码所有样本...")
    inputs = [
        f"question: {q} context: {c}" 
        for q, c in zip(questions, contexts)
    ]
    
    encoded = tokenizer(
        inputs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    print(f"✅ 编码完成！")
    print(f"   - 样本数量: {len(inputs)}")
    print(f"   - input_ids 形状: {encoded['input_ids'].shape}")
    print(f"   - attention_mask 形状: {encoded['attention_mask'].shape}")

except ImportError as e:
    print(f"❌ 需要安装 transformers: pip install transformers")
    print(f"   错误: {e}")
    encoded = None

⏳ 正在加载 T5 分词器...
📦 检测到预训练权重，使用 t5-base 架构
✅ 分词器加载成功: t5-base

🔤 分词器编码示例:
   输入: question: Classify the network packet... context: 7b89 9867 5bcf 9062 13b1 bf05 939b c088 ...
   Token数量(前15): ['▁question', ':', '▁Class', 'ify', '▁the', '▁network', '▁packet', '...', '▁context', ':', '▁7', 'b', '89', '▁', '98']

🔄 批量编码所有样本...
✅ 编码完成！
   - 样本数量: 500
   - input_ids 形状: torch.Size([500, 512])
   - attention_mask 形状: torch.Size([500, 512])


In [34]:
# 基于训练标签映射，对测试集进行分词编码
if encoded is not None and tokenizer is not None and label_names:
    print("⏳ 正在编码测试集以与训练标签保持一致...")
    payload_cols_sorted = sorted(payload_cols, key=lambda x: int(x.split("_")[-1]))
    label_map = {name: idx for idx, name in enumerate(label_names)}

    # 过滤掉标签不在训练集映射中的测试样本
    labels_raw = df_test["attack_label"].map(label_map)
    mask_known = labels_raw.notna()
    if not mask_known.all():
        unknown_count = (~mask_known).sum()
        print(f"⚠️  有 {unknown_count} 条测试样本标签未在训练标签中，已跳过。")

    df_test_filtered = df_test.loc[mask_known].reset_index(drop=True)
    labels_test = labels_raw.loc[mask_known].astype(np.int64).to_numpy()

    def df_to_hex_contexts(df_src):
        byte_array = df_src[payload_cols_sorted].to_numpy(dtype=np.uint16)
        contexts_out = []
        for row in byte_array:
            hex_str = "".join(f"{int(b):02x}" for b in row)
            contexts_out.append(" ".join(hex_str[i:i+4] for i in range(0, len(hex_str), 4)))
        return contexts_out

    question_template = "Classify the network packet"
    contexts_test = df_to_hex_contexts(df_test_filtered)
    questions_test = [question_template] * len(contexts_test)

    inputs_test = [f"question: {q} context: {c}" for q, c in zip(questions_test, contexts_test)]
    encoded_test = tokenizer(
        inputs_test,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    print("✅ 测试集编码完成！")
    print(f"   - 测试样本数: {len(df_test_filtered)} (原始: {len(df_test)})")
    print(f"   - input_ids 形状: {encoded_test['input_ids'].shape}")
    print(f"   - 标签覆盖的类别数: {len(np.unique(labels_test))} / {len(label_names)}")
else:
    encoded_test, labels_test = None, None
    print("⚠️  跳过测试集编码（缺少 tokenizer/encoded 或标签映射）")

⏳ 正在编码测试集以与训练标签保持一致...
✅ 测试集编码完成！
   - 测试样本数: 500 (原始: 500)
   - input_ids 形状: torch.Size([500, 30])
   - 标签覆盖的类别数: 2 / 2


## 7. 加载 T5 编码器和分类头

加载预训练的 T5 模型作为编码器，并添加一个线性分类头。

In [35]:
if encoded is not None:
    try:
        from transformers import T5ForConditionalGeneration
        from torch import nn
        from pathlib import Path
        
        # 预训练权重路径
        weights_path = Path("../models/weights.pth")
        
        print("🤖 加载 T5 模型...")
        model = T5ForConditionalGeneration.from_pretrained(model_name)
        
        # 加载预训练权重
        if weights_path.exists():
            print(f"📦 找到预训练权重: {weights_path}")
            print("   正在加载 PCAP_encoder 预训练权重...")
            try:
                # 尝试使用 weights_only=True (PyTorch 2.0+)
                state = torch.load(weights_path, map_location="cpu", weights_only=True)
            except TypeError:
                # 兼容旧版 PyTorch
                state = torch.load(weights_path, map_location="cpu")
            
            # 加载权重（strict=False 允许部分加载）
            missing, unexpected = model.load_state_dict(state, strict=False)
            
            if len(missing) == 0:
                print("   ✅ 所有权重加载成功！")
            else:
                print(f"   ⚠️  部分权重已加载 (缺少 {len(missing)} 个键)")
            
            use_pretrained = True
        else:
            print(f"⚠️  未找到预训练权重: {weights_path}")
            print("   使用随机初始化权重（仅用于演示流程）")
            use_pretrained = False
        
        print()
        
        # 提取编码器
        encoder = model.encoder.to(device)
        
        # 冻结参数
        for p in encoder.parameters():
            p.requires_grad = False
        
        hidden_size = model.config.d_model
        
        print(f"✅ 编码器加载成功！")
        print(f"   - 隐藏维度: {hidden_size}")
        print(f"   - 编码器层数: {model.config.num_layers}")
        print(f"   - 注意力头数: {model.config.num_heads}")
        print(f"   - 设备: {device}")
        print(f"   - 使用预训练权重: {'是 ✅' if use_pretrained else '否 ❌'}")
        print()
        
        # 创建分类头
        num_classes = len(label_names)
        head = nn.Linear(hidden_size, num_classes).to(device)
        
        print(f"✅ 分类头创建成功！")
        print(f"   - 输入维度: {hidden_size}")
        print(f"   - 输出类别数: {num_classes}")
        print(f"   - 参数量: {sum(p.numel() for p in head.parameters()):,}")
        
        if use_pretrained:
            print()
            print("💡 提示: 编码器使用预训练权重，但分类头是随机初始化的")
            print("   真正的分类性能需要在下游任务上微调分类头")
        
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        import traceback
        traceback.print_exc()
        encoder, head = None, None
        use_pretrained = False
else:
    encoder, head = None, None
    use_pretrained = False

🤖 加载 T5 模型...
📦 找到预训练权重: ../models/weights.pth
   正在加载 PCAP_encoder 预训练权重...
   ⚠️  部分权重已加载 (缺少 1 个键)

✅ 编码器加载成功！
   - 隐藏维度: 768
   - 编码器层数: 12
   - 注意力头数: 12
   - 设备: cuda
   - 使用预训练权重: 是 ✅

✅ 分类头创建成功！
   - 输入维度: 768
   - 输出类别数: 2
   - 参数量: 1,538

💡 提示: 编码器使用预训练权重，但分类头是随机初始化的
   真正的分类性能需要在下游任务上微调分类头


## 8. 推理与分类预测

通过编码器提取数据包的表示向量，然后用分类头进行预测。

In [37]:
# 训练线性分类头以获得真正的预测能力（冻结编码器）
missing_vars = [name for name in ["encoded", "encoder", "head", "labels"] if name not in globals()]
if missing_vars:
    print(f"⚠️  缺少必要变量: {missing_vars}，请先按顺序运行分词和模型加载相关的单元。")
elif encoded is None or encoder is None or head is None:
    print("⚠️  无法训练分类头（编码器或分词结果不存在）")
else:
    from torch import nn
    from torch.utils.data import DataLoader, TensorDataset
    import torch.optim as optim

    print("🧪 开始在 500 条真实样本上训练分类头 (冻结编码器)...")

    # 准备数据张量
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)
    label_tensor = torch.tensor(labels, device=device)

    dataset = TensorDataset(input_ids, attention_mask, label_tensor)
    batch_size = 8 if device == "cuda" else 16
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    head.train()
    encoder.eval()  # 编码器冻结，仅前向

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(head.parameters(), lr=1e-3)
    epochs = 20

    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        correct = 0
        total = 0
        for batch_input_ids, batch_attention_mask, batch_labels in loader:
            optimizer.zero_grad()
            with torch.no_grad():
                outputs = encoder(input_ids=batch_input_ids, attention_mask=batch_attention_mask, return_dict=True)
                hidden = outputs.last_hidden_state[:, 0, :]
            logits = head(hidden)
            loss = criterion(logits, batch_labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * batch_labels.size(0)
            preds_batch = logits.argmax(dim=1)
            correct += (preds_batch == batch_labels).sum().item()
            total += batch_labels.size(0)

        avg_loss = epoch_loss / total
        acc = correct / total * 100
        print(f"   Epoch {epoch}/{epochs} - loss: {avg_loss:.4f}, acc: {acc:.2f}%")

    head.eval()
    print("✅ 分类头训练完成！")


🧪 开始在 500 条真实样本上训练分类头 (冻结编码器)...
   Epoch 1/20 - loss: 0.0613, acc: 98.20%
   Epoch 2/20 - loss: 0.0563, acc: 98.40%
   Epoch 3/20 - loss: 0.0531, acc: 98.20%
   Epoch 4/20 - loss: 0.0504, acc: 98.20%
   Epoch 5/20 - loss: 0.0490, acc: 98.20%
   Epoch 6/20 - loss: 0.0470, acc: 98.20%
   Epoch 7/20 - loss: 0.0453, acc: 98.40%
   Epoch 8/20 - loss: 0.0443, acc: 98.40%
   Epoch 9/20 - loss: 0.0438, acc: 98.40%
   Epoch 10/20 - loss: 0.0424, acc: 98.40%
   Epoch 11/20 - loss: 0.0416, acc: 98.00%
   Epoch 12/20 - loss: 0.0417, acc: 98.20%
   Epoch 13/20 - loss: 0.0403, acc: 98.20%
   Epoch 14/20 - loss: 0.0396, acc: 98.40%
   Epoch 15/20 - loss: 0.0396, acc: 98.20%
   Epoch 16/20 - loss: 0.0385, acc: 98.60%
   Epoch 17/20 - loss: 0.0389, acc: 98.40%
   Epoch 18/20 - loss: 0.0384, acc: 98.80%
   Epoch 19/20 - loss: 0.0372, acc: 98.80%
   Epoch 20/20 - loss: 0.0380, acc: 98.40%
✅ 分类头训练完成！


In [38]:
if encoder is not None and head is not None and encoded_test is not None and labels_test is not None:
    print("🧠 开始在测试集上推理...")
    print()

    input_ids = encoded_test["input_ids"]
    attention_mask = encoded_test["attention_mask"]
    batch_size = 8 if device == "cuda" else input_ids.size(0)
    num_samples = input_ids.size(0)
    num_batches = (num_samples + batch_size - 1) // batch_size
    print(f"   - 每批大小: {batch_size}")
    print(f"   - 总样本数: {num_samples}")
    print(f"   - 批次数: {num_batches}")
    
    with torch.no_grad():
        print("   1️⃣  通过 T5 编码器提取表示并按批次推理...")
        probs_list = []
        
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_input_ids = input_ids[start:end].to(device)
            batch_attention_mask = attention_mask[start:end].to(device)

            outputs = encoder(input_ids=batch_input_ids, attention_mask=batch_attention_mask, return_dict=True)
            hidden = outputs.last_hidden_state[:, 0, :]
            if start == 0:
                print(f"      ✅ 隐藏表示形状: {hidden.shape}")
                print(f"      表示维度: {hidden.shape[1]}")
                print()

            logits = head(hidden)
            if start == 0:
                print("   2️⃣  通过分类头预测类别...")
                print(f"      ✅ Logits 形状: {logits.shape}")
                print()

            batch_probs = torch.softmax(logits, dim=1).cpu().numpy()
            probs_list.append(batch_probs)

            del batch_input_ids, batch_attention_mask, outputs, hidden, logits
            if device == "cuda":
                torch.cuda.empty_cache()
        
        print("   3️⃣  计算预测概率...")
        probs_test = np.concatenate(probs_list, axis=0)
        preds_test = probs_test.argmax(axis=1)
        print("      ✅ 测试集预测完成！")

    print()
    print("="*70)
    print("✅ 测试集推理流程完成！")
    print("="*70)
else:
    print("⚠️  跳过测试集推理（缺少模型或 encoded_test/labels_test）")
    preds_test, probs_test = None, None

🧠 开始在测试集上推理...

   - 每批大小: 8
   - 总样本数: 500
   - 批次数: 63
   1️⃣  通过 T5 编码器提取表示并按批次推理...
      ✅ 隐藏表示形状: torch.Size([8, 768])
      表示维度: 768

   2️⃣  通过分类头预测类别...
      ✅ Logits 形状: torch.Size([8, 2])

   3️⃣  计算预测概率...
      ✅ 测试集预测完成！

✅ 测试集推理流程完成！


## 9. 结果分析与总结

查看模型的预测结果，并计算准确率。

In [39]:
if preds_test is not None and probs_test is not None and labels_test is not None and len(labels_test) > 0:
    print("="*80)
    print("📊 测试集预测结果详情")
    if use_pretrained:
        print("   [使用预训练 PCAP_encoder 权重]")
    else:
        print("   [使用随机初始化权重]")
    print("="*80)
    print()
    print(f"{'样本':^6} │ {'真实标签':^15} │ {'预测标签':^15} │ {'置信度':^10} │ {'状态':^6}")
    print(f"{'─'*6}─┼─{'─'*15}─┼─{'─'*15}─┼─{'─'*10}─┼─{'─'*6}")
    
    correct = 0
    for i in range(len(labels_test)):
        true_label = label_names[labels_test[i]]
        pred_label = label_names[preds_test[i]]
        confidence = probs_test[i].max() * 100
        is_correct = "✅" if labels_test[i] == preds_test[i] else "❌"
        if labels_test[i] == preds_test[i]:
            correct += 1
        
        print(f"{i+1:^6} │ {true_label:^15} │ {pred_label:^15} │ {confidence:^10.1f}% │ {is_correct:^6}")
    
    accuracy = correct / len(labels_test) * 100
    print(f"{'─'*6}─┴─{'─'*15}─┴─{'─'*15}─┴─{'─'*10}─┴─{'─'*6}")
    print()
    print(f"📈 测试集准确率: {accuracy:.1f}% ({correct}/{len(labels_test)})")
    print()
    
    # 混淆矩阵
    print("="*80)
    print("📊 测试集混淆矩阵")
    print("="*80)
    from collections import Counter
    
    confusion = {true: {pred: 0 for pred in label_names} for true in label_names}
    for true_idx, pred_idx in zip(labels_test, preds_test):
        true_name = label_names[true_idx]
        pred_name = label_names[pred_idx]
        confusion[true_name][pred_name] += 1
    
    # 打印混淆矩阵
    print(f"\n{'真实/预测':>15}", end="")
    for pred_name in label_names:
        print(f"{pred_name:>15}", end="")
    print()
    print("─" * (15 + 15 * len(label_names)))
    
    for true_name in label_names:
        print(f"{true_name:>15}", end="")
        for pred_name in label_names:
            count = confusion[true_name][pred_name]
            print(f"{count:>15}", end="")
        print()
    
    print()
    print("="*80)
    print("💡 结果分析")
    print("="*80)
    
    if use_pretrained:
        print("✅ 当前使用的是 PCAP_encoder 预训练权重！")
        print()
        print("📌 关于预测结果的说明：")
        print("   - 编码器已使用预训练权重，可以提取有意义的数据包表示")
        print("   - 但分类头是随机初始化的，需要微调才能得到好的分类效果")
        print("   - 预训练权重学习的是 QA 和 Denoiser 任务，不是分类任务")
        print()
        print("🔬 预训练权重的价值：")
        print("   1. 编码器已学会理解十六进制数据包格式")
        print("   2. 提取的表示向量包含协议语义信息")
        print("   3. 在分类任务上微调可以获得更好的收敛速度和性能")
        print()
        print("📊 如果准确率仍然接近随机，这是因为：")
        print("   - 分类头未经训练，只是随机线性层")
        print("   - 需要在分类数据集上微调分类头")
    else:
        print("⚠️  当前使用的是随机初始化的权重，准确率接近随机猜测是正常的！")
        print()
        print("要获得真实的性能评估，需要：")
        print("   1. 使用在 QA 和 Denoiser 任务上预训练的 PCAP_encoder 权重")
        print("   2. 在大规模网络流量数据集上进行微调")
        print("   3. 使用完整的数据包（不只是前64字节）")
    
    print()
    print("本演示的目的是展示完整的工作流程。")
    
else:
    print("⚠️  未执行测试集推理，跳过结果分析")

📊 测试集预测结果详情
   [使用预训练 PCAP_encoder 权重]

  样本   │      真实标签       │      预测标签       │    置信度     │   状态  
───────┼─────────────────┼─────────────────┼────────────┼───────
  1    │     BENIGN      │     BENIGN      │    98.3   % │   ✅   
  2    │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  3    │     BENIGN      │     BENIGN      │    98.2   % │   ✅   
  4    │     BENIGN      │     BENIGN      │    99.6   % │   ✅   
  5    │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  6    │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  7    │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  8    │     BENIGN      │     BENIGN      │    99.9   % │   ✅   
  9    │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  10   │     BENIGN      │     BENIGN      │   100.0   % │   ✅   
  11   │     BENIGN      │     BENIGN      │    99.9   % │   ✅   
  12   │     BENIGN      │     BENIGN      │    99.9   % │   ✅   
  13   │     BENIGN      │     BENIGN 

# 总结

## 🎯 本 Notebook 演示的完整流程

### 数据处理流程
1. ✅ **数据加载**: 从 CIC-IDS2017 Parquet 文件加载真实流量数据
2. ✅ **数据可视化**: 查看真实数据的标签分布和统计信息
3. ✅ **格式转换**: 将字节数据转换为十六进制字符串（every4 格式）

### 模型推理流程
4. ✅ **构建输入**: 创建 "question + context" 格式的输入
5. ✅ **分词编码**: 使用 T5 分词器将文本转换为 token IDs
6. ✅ **编码器提取**: 通过 T5 编码器提取 512 维表示向量
7. ✅ **分类预测**: 使用线性分类头进行攻击类型预测
8. ✅ **结果分析**: 展示预测结果、准确率和混淆矩阵

---

## 🔑 关键技术点

### 1. NLP 技术应用于网络安全
- 将网络数据包视为"文本"
- 使用预训练语言模型（T5）理解网络协议
- Text-to-Text 框架统一处理多种任务

### 2. 自监督学习
- **QA 任务**: 从十六进制中提取字段值
- **Denoiser 任务**: 重建被破坏的数据包
- 两个任务共同预训练，学习通用的数据包表示

### 3. 迁移学习
- 预训练阶段学习协议结构和特征
- 微调阶段适配特定的分类任务
- 表示向量可用于多种下游任务

---

## 📚 后续学习建议

### 深入了解
1. 阅读 PCAP_encoder 论文了解理论基础
2. 查看 `Preprocess/` 目录学习数据预处理
3. 研究 `Core/classes/` 中的模型实现

### 实践建议
1. 使用更多真实 PCAP 文件测试
2. 尝试不同的十六进制格式（every2, every4, noSpace）
3. 在预训练权重上进行微调

### 扩展方向
1. 异常检测：使用表示向量检测异常流量
2. 协议识别：识别未知的网络协议
3. 流量生成：使用 T5 的生成能力合成流量

---

## 🙏 致谢

本演示基于以下工作：
- **PCAP_encoder**: Giovanni Dettori 的博士论文
- **T5**: Google 的 Text-to-Text Transfer Transformer
- **Transformers**: HuggingFace 的开源库

---

## 📞 需要帮助？

如果遇到问题，可以：
1. 检查依赖是否正确安装
2. 查看原始的 `demo_minimal_data.py` 和 `demo_pipeline.py`
3. 参考项目的 README.md 文档
